# LLM 

In [9]:
request_content = """Dal PDF fornito, estrai i valori di F1, F2, F3 e restituiscili esclusivamente in formato JSON. 
        Il JSON deve avere esattamente questa struttura:
        {
            "f1_kwh": x_kwh,
            "f2_kwh": y_kwh,
            "f3_kwh": z_kwh,
            "f1_kvarh_capacitive": x1_kvarh,
            "f1_kvarh_inductive": x2_kvarh,
            "f1_kvarh_injected": x3_kvarh,
            "f2_kvarh_capacitive": y1_kvarh,
            "f2_kvarh_inductive": y2_kvarh,
            "f2_kvarh_injected": y3_kvarh,
            "f3_kvarh_capacitive": z1_kvarh,
            "f3_kvarh_inductive": z2_kvarh,
            "f3_kvarh_injected": z3_kvarh,
            "error_msg": ""
        }
        Regole di estrazione dei dati:
        * f*_kwh:
            * Prendi i valori dalla colonna "Quantità" nella sezione "Consumo fatturato" del PDF.
            * Ogni fascia (F1, F2, F3) deve avere il proprio valore.
        * f*_kvarh_capacitive, f*_kvarh_inductive, f*_kvarh_injected:
            * Estrarre i valori direttamente dalle seguenti sezioni del PDF per ciascuna fascia:
                * f*_kvarh_capacitive → Consumo rilevato energia reattiva capacitiva
                * f*_kvarh_inductive → Consumo rilevato energia reattiva induttiva
                * f*_kvarh_injected → Consumo fatturato energia reattiva immessa
            * Non eseguire somme o calcoli: usa il valore riportato nel PDF.
            * Tutti i valori devono corrispondere alla stessa fascia oraria (F1, F2, F3).
        Se uno o più dati non sono reperibili, restituisci tutti i campi null, con questo JSON:
        {
            "f1_kwh": null,
            "f2_kwh": null,
            "f3_kwh": null,
            "f1_kvarh_capacitive": null,
            "f1_kvarh_inductive": null,
            "f1_kvarh_injected": null,
            "f2_kvarh_capacitive": null,
            "f2_kvarh_inductive": null,
            "f2_kvarh_injected": null,
            "f3_kvarh_capacitive": null,
            "f3_kvarh_inductive": null,
            "f3_kvarh_injected": null,
            "error_msg": "FAILED TO READ CONSUMPTIONS FROM ALLEDGED BILL FILE"
        }
        ATTENZIONE:
        * La risposta deve contenere solo il JSON, nessun commento, spiegazione o testo aggiuntivo
        * I nomi dei campi devono essere esattamente quelli indicati sopra"""

### AUTENTIFICAZIONE

In [2]:
import requests


BASE_URL = "http://localhost:3000"
url = f"{BASE_URL}/api/v1/auths/signin"
payload = {
    "email": "eleonora.fabris@dbagroup.it", 
    "password": "Scoiattolo1!"
}
response = requests.post(url, json=payload)
token = response.json()["token"]


In [3]:
url = f"{BASE_URL}/api/v1/models"
JWT_TOKEN = token

HEADERS_JSON = {
    "Authorization": f"Bearer {JWT_TOKEN}",
    "Content-Type": "application/json"
}
HEADERS_FILE = {
    "Authorization": f"Bearer {JWT_TOKEN}"
}

response = requests.get(url, headers=HEADERS_JSON)

if response.status_code == 200:
    print("Autenticazione riuscita!")
    print(response.json()) # lista modelli disponibili
else:
    raise Exception(f"Errore nell'autenticazione: {response.status_code}")



Autenticazione riuscita!
{'data': [{'id': 'gemma3:4b', 'name': 'gemma3:4b', 'object': 'model', 'created': 1766505451, 'owned_by': 'ollama', 'ollama': {'name': 'gemma3:4b', 'model': 'gemma3:4b', 'modified_at': '2025-12-22T16:59:15.153912+01:00', 'size': 3338801804, 'digest': 'a2af6cc3eb7fa8be8504abaf9b04e88f17a119ec3f04a3addf55f92841195f5a', 'details': {'parent_model': '', 'format': 'gguf', 'family': 'gemma3', 'families': ['gemma3'], 'parameter_size': '4.3B', 'quantization_level': 'Q4_K_M'}, 'connection_type': 'local', 'urls': [0]}, 'connection_type': 'local', 'tags': [], 'actions': [], 'filters': []}, {'id': 'falcon3:7b', 'name': 'falcon3:7b', 'object': 'model', 'created': 1766505451, 'owned_by': 'ollama', 'ollama': {'name': 'falcon3:7b', 'model': 'falcon3:7b', 'modified_at': '2025-12-22T16:51:27.8742221+01:00', 'size': 4569738977, 'digest': '472ea1c89f64f932368963bfb8c7f54bd797f2161932cf0a72766a587b2e586d', 'details': {'parent_model': '', 'format': 'gguf', 'family': 'llama', 'families

### CREAZIONE NUOVA CHAT

In [ ]:
import requests
import uuid
import time


initial_assistant_id = str(uuid.uuid4())

payload_chat = {
    "chat": {
        "name": "Lettura della bolletta",
        "system_prompt": "Rispondi solo in JSON",
        "history": {
            "messages": [
                {
                    "id": initial_assistant_id,
                    "parentId": None,
                    "role": "assistant",
                    "content": "",     
                    "model": "falcon3:7b",
                    "models": ["falcon3:7b"],
                    "timestamp": int(time.time())
                }
            ],
            "currentId": initial_assistant_id
        }
    }
}

url_new_chat = f"{BASE_URL}/api/v1/chats/new"
response = requests.post(url_new_chat, json=payload_chat, headers=HEADERS_JSON)
if response.status_code not in [200, 201]:
    raise Exception(f"Errore nella creazione chat: {response.status_code} - {response.text}")

chat_data = response.json()
chat_id = chat_data["id"]
current_id = initial_assistant_id
print("Nuova chat creata:", chat_id)


Nuova chat creata: 270a7770-ae9d-4f07-a98f-30a573c3ea53


### CARICAMENTO FILE

In [ ]:
file_path = "C:\DEV\LAB\electrical-energy-billing\Fattura_0000005222062506.pdf"
url_upload_file = f"{BASE_URL}/api/v1/files/?process=true"

with open(file_path, "rb") as f:
    files = {"file": f} 
    response = requests.post(url_upload_file, files=files, headers=HEADERS_FILE)
if response.status_code not in [200, 201]:
    raise Exception(f"Errore nel caricamento del file: {response.status_code} - {response.text}")

file_data = response.json()
file_id = file_data.get("id")
print("File caricato:", file_id)

File caricato: 7d46a2b2-d928-4c3b-b358-c66b69a046fa


In [12]:
user_msg_id = str(uuid.uuid4())
assistant_msg_id = str(uuid.uuid4())

messages = [
    {
        "id": user_msg_id,
        "parentId": current_id,
        "role": "user",
        "content": request_content,
        "models": ["falcon3:7b"],
        "timestamp": int(time.time())
    },
    {
        "id": assistant_msg_id,
        "parentId": user_msg_id,
        "role": "assistant",
        "content": "", # risposta del modello
        "model": "falcon3:7b",
        "models": ["falcon3:7b"],
        "timestamp": int(time.time())
    }
]

current_id = assistant_msg_id # aggiorna per la prossima richiesta

payload_prompt = {
    "chat_id": chat_id,       
    "model": "falcon3:7b",
    "messages": messages,
    "files": [{"id": file_id}], # associa il file alla chat
    "params": {
        "max_new_tokens": 500, 
        "temperature": 0.7, # se 0.0 risponde sempre allo stesso modo, estremamente preciso
        "top_p": 0.9 # indica la variabilità del vocabolario
    }
}

In [7]:
# url_new_chat = f"{BASE_URL}/api/v1/chats/new"
# payload_chat = {
#   "chat": {
#     "name": "Lettura della bolletta",
#     "system_prompt": "Rispondi solo in JSON"
#   }
# }
# response = requests.post(url_new_chat, json=payload_chat, headers=HEADERS)
# if response.status_code in [200, 201]:
#     chat_data = response.json()
#     chat_id = chat_data.get("id")
#     print("Nuova chat creata:", chat_id)
# else:
#     print("Errore nella creazione chat:", response.status_code, response.text)


In [8]:
# file_path = "C:\DEV\LAB\electrical-energy-billing\Fattura_0000005222062506.pdf"
# url_upload_file = f"{BASE_URL}/api/v1/files/?process=true"

# with open(file_path, "rb") as f:
#     files = {"file": f} 
#     response = requests.post(url_upload_file, files=files, headers=HEADERS_FILE)
#     file_data = response.json()
#     file_id = file_data.get("id")
# if response.status_code in [200, 201]:
#     print("File caricato correttamente!")
#     print(file_data) 
# else:
#     print(f"Errore upload file: {response.status_code} - {response.text}")

### INVIO DEL PROMPT

In [13]:
url_prompt = f"{BASE_URL}/api/chat/completions"

response = requests.post(url_prompt, json=payload_prompt, headers=HEADERS_JSON)
if response.status_code != 200:
    raise Exception(f"Errore completions: {response.status_code} - {response.text}")

data = response.json()
task_id = data.get("task_id")
if not task_id:
    raise Exception("Task ID non trovato.")


Exception: Errore completions: 400 - {"detail":"'NoneType' object has no attribute 'get'"}

### ELIMINA CHAT

In [44]:
url_delete_chat = f"{BASE_URL}/api/v1/chats/{chat_id}"

response = requests.delete(url_delete_chat, headers=HEADERS_FILE)
if response.status_code in [200, 204]: # cancellazione riuscita: risposta JSON opzionale, nessun contenuto restituito
    print(f"Chat {chat_id} cancellata correttamente.")
else:
    print(f"Errore nella cancellazione: {response.status_code} - {response.text}")


Chat c8e3bfcc-4a23-4349-be9e-a82e9da19a1f cancellata correttamente.
